In [1]:
import sys
sys.path.insert(0, '../ELINA/python_interface/')
import numpy as np
import logging
import re
import csv
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
from gurobipy import *
import time

%run utils.ipynb

libc = CDLL(find_library('c'))

cstdout = c_void_p.in_dll(libc, 'stdout')

In [2]:
def optimize_bounds_for_objective(objective, lb_affine_neuron, 
                                  ub_affine_neuron,affine_bounds_for_layer, 
                                  slope_intercept_for_layer,neuron_num,layer_num, nn):
    '''
    function gets upper or lower bounds of ELINA for a single neuron in layer and uses solver to optimize them
    '''
    

    layers_count = len(affine_bounds_for_layer.keys())
    neurons_count = len(affine_bounds_for_layer['layer0'])        #same n neurons per layer

    m = Model("mip1")
    m.Params.outputflag=0
    m.setParam('TimeLimit',3*60)
    
    weights = nn.weights
    bias = nn.biases

    #
    h = {}
    h_relu ={}
    for layer in range(layer_num+1):
        if layer == layer_num:
            optimize_to_neuron = neuron_num+1
        else:
            optimize_to_neuron = neurons_count

        for neuron in range(optimize_to_neuron):
            
            h[layer,neuron] = m.addVar(vtype= GRB.CONTINUOUS, 
                                           lb=affine_bounds_for_layer['layer'+str(layer)][neuron,0],
                                           ub= affine_bounds_for_layer['layer'+str(layer)][neuron,1],
                                      name = 'h[%s,%s]'%(layer,neuron))
                
            h_relu[layer,neuron] = m.addVar(vtype= GRB.CONTINUOUS,
                                           name = 'h_relu[%s,%s]'%(layer,neuron))
    m.update()
    
    
    # Set objective
    if objective == 'minimize':
        m.setObjective(h_relu[layer_num,neuron_num],GRB.MINIMIZE)

    if objective == 'maximize':
        m.setObjective(h_relu[layer_num,neuron_num],GRB.MAXIMIZE)

    m.update()
    
    for layer in range(layer_num+1):

        
        if layer == layer_num:
            optimize_to_neuron = neuron_num + 1
        else:
            optimize_to_neuron = neurons_count
    
        for neuron in range(optimize_to_neuron):
            
            
            lb = affine_bounds_for_layer['layer'+str(layer)][neuron,0]
            ub = affine_bounds_for_layer['layer'+str(layer)][neuron,1]
            
            if layer > 0:
                h_relu_prev_layer = [h_relu[layer-1,i] for i in range(neurons_count)]
                z = LinExpr(weights[layer][neuron,:].reshape(neurons_count,1),h_relu_prev_layer)
                z.addConstant(bias[layer][neuron])            

                m.addConstr(rhs = z, sense = GRB.EQUAL, lhs = h[layer,neuron],name='linear[%s,%s]'%(layer,neuron))

            else:
                m.addConstr(lhs = h[layer,neuron], rhs= affine_bounds_for_layer['layer0'][neuron,0],
                            sense= GRB.GREATER_EQUAL,name='lb_h[0,%s]'%(neuron))
                m.addConstr(lhs = h[layer,neuron], rhs= affine_bounds_for_layer['layer0'][neuron,1],
                            sense= GRB.LESS_EQUAL,name='ub_h[0,%s]'%(neuron))

            if (lb <= 0) & (ub <= 0):
                m.addConstr(lhs = h_relu[layer,neuron], rhs=0.0, sense= GRB.EQUAL)
                
            if (lb >0) & (ub > 0):
                m.addConstr(lhs = h_relu[layer,neuron], rhs=h[layer,neuron], sense= GRB.EQUAL)
                
            
            if (lb<0) & (ub>0):
                
                m.addConstr(h_relu[layer,neuron] <= slope_intercept_for_layer['layer'+str(layer)][neuron,0]*
                                h[layer,neuron] + slope_intercept_for_layer['layer'+str(layer)][neuron,1],
                               name='line_h_relu[%s,%s]'%(layer,neuron))

            m.addConstr(h_relu[layer,neuron]>=0,name='pos_h_relu[%s,%s]'%(layer,neuron))
            m.addConstr(h_relu[layer,neuron]>= h[layer,neuron], name='gr_h_relu[%s,%s]'%(layer,neuron))
                
            
                   
                
    m.update()
    
    
#     m.write("model.mps")

    
    m.optimize()
    
    if (GRB.OPTIMAL !=2):
        print("\nGurobi Status Code:", GRB.OPTIMAL)

    
    return m.objVal


    


In [3]:
def get_bounds_linear_solver(nn,affine_bounds_for_layer,apply_linear_solver_to_layer,neuron_num, slope_intercept_for_layer):
    
    
    '''function gets all previous bounds and lambda/mu, returns the bounds for specific neuron n in layer m'''

    lb_affine = affine_bounds_for_layer["layer"+str(apply_linear_solver_to_layer)][neuron_num][0]
    ub_affine = affine_bounds_for_layer["layer"+str(apply_linear_solver_to_layer)][neuron_num][1]
    

    h_relu_ub = optimize_bounds_for_objective('maximize', lb_affine, 
                                              ub_affine,
                                              affine_bounds_for_layer,
                                              slope_intercept_for_layer,neuron_num, 
                                              apply_linear_solver_to_layer,
                                             nn)
    h_relu_lb = optimize_bounds_for_objective('minimize', lb_affine, 
                                              ub_affine,affine_bounds_for_layer,
                                              slope_intercept_for_layer,neuron_num, 
                                              apply_linear_solver_to_layer,
                                             nn)
        
        
        
    return h_relu_lb, h_relu_ub

    

In [4]:
def predict_label(nn, LB_N0, UB_N0, label):
    num_pixels = len(LB_N0)  # number of pixels in image

    nn.ffn_counter = 0

    numlayer = nn.numlayer

    logging.debug("Layers #:{}".format(numlayer))

    elina_manager = elina_box_manager_alloc()  # create manager to the box domain

    interval_array = elina_interval_array_alloc(num_pixels)  # allocate a new ELina interval

    for i in range(num_pixels):  # disturb over the interval

        elina_interval_set_double(interval_array[i], LB_N0[i], UB_N0[i])

    abstraction_shape = elina_abstract0_of_box(elina_manager, 0, num_pixels, interval_array)  # create elina_element

    elina_interval_array_free(interval_array, num_pixels)
    
    affine_bounds_for_layer ={}
    slope_intercept_for_layer = {}
    solver_bounds_for_layer = {}


    for layer_num in range(numlayer):


        if nn.layertypes[layer_num] in ['ReLU', 'Affine']:

            weights = nn.weights[nn.ffn_counter]

            biases = nn.biases[nn.ffn_counter]

            dims = elina_abstract0_dimension(elina_manager, abstraction_shape)

            num_in_pixels = dims.intdim + dims.realdim

            num_out_pixels = len(weights)  # len(weights) = number of neurons in layer

            dimadd = elina_dimchange_alloc(0, num_out_pixels)  # allocate new Elina dim change

            for i in range(num_out_pixels):
                dimadd.contents.dim[i] = num_in_pixels

            elina_abstract0_add_dimensions(elina_manager, True, abstraction_shape, dimadd, False)

            elina_dimchange_free(dimadd)

            np.ascontiguousarray(weights, dtype=np.double)

            np.ascontiguousarray(biases, dtype=np.double)

            var = num_in_pixels

            print("original num_pixels: {} , num_in_pixels: {}, num_out_pixels:{}".format(num_pixels,

                                                                                                  num_in_pixels,

                                                                                                  num_out_pixels))

            #--------------------------- handle affine layer---------------------------
                
            for i in range(num_out_pixels): 

                tdim = ElinaDim(var)
                linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
                abstraction_shape = elina_abstract0_assign_linexpr_array(elina_manager, True, abstraction_shape, tdim,
                                                                         linexpr0, 1, None)
                var += 1

            dimrem = elina_dimchange_alloc(0, num_in_pixels)

            for i in range(num_in_pixels):
                dimrem.contents.dim[i] = i

            elina_abstract0_remove_dimensions(elina_manager, True, abstraction_shape, dimrem)  
            elina_dimchange_free(dimrem)  


            bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)

            #--------------------------- handle RELU ---------------------------

            if (nn.layertypes[layer_num] == 'ReLU'):
                print('---------------RELU--------------')

                abstraction_shape = relu_box_layerwise(elina_manager, True, abstraction_shape, 0, num_out_pixels)

                bounds = elina_abstract0_to_box(elina_manager, abstraction_shape) 

                element_dim = elina_abstract0_dimension(elina_manager, abstraction_shape)
                    

#                     for i in range(num_out_pixels):
#                         print(
#                             "[{},{}]".format(bounds[i].contents.inf.contents.val.dbl,

#                                                                               bounds[i].contents.sup.contents.val.dbl))

            nn.ffn_counter += 1


        else:

            print(' net type not supported')


    dims = elina_abstract0_dimension(elina_manager, abstraction_shape)

    output_size = dims.intdim + dims.realdim

    bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)


    verified_flag = True

    predicted_label = 0


    for i in range(output_size):

        inf = bounds[i].contents.inf.contents.val.dbl

        flag = True

        for j in range(output_size):

            if (j != i):

                sup = bounds[j].contents.sup.contents.val.dbl

                if (inf <= sup):
                    flag = False

                    break

        if (flag):
            predicted_label = i

            break


    elina_interval_array_free(bounds, output_size)

    elina_abstract0_free(elina_manager, abstraction_shape)

    elina_manager_free(elina_manager)

    return predicted_label

In [5]:
def verify(nn, LB_N0, UB_N0, label,apply_linear_solver_to_layer):
    num_pixels = len(LB_N0)  
    nn.ffn_counter = 0
    numlayer = nn.numlayer

    elina_manager = elina_box_manager_alloc()  # create manager to the box domain

    interval_array = elina_interval_array_alloc(num_pixels)  # allocate a new ELina interval

    for i in range(num_pixels):  

        elina_interval_set_double(interval_array[i], LB_N0[i], UB_N0[i])

    ## construct input abstraction
    abstraction_shape = elina_abstract0_of_box(elina_manager, 0, num_pixels, interval_array)  
    elina_interval_array_free(interval_array, num_pixels)
    
    affine_bounds_for_layer ={}
    slope_intercept_for_layer = {}
    solver_bounds_for_layer = {}


    for layer_num in range(numlayer):

        if nn.layertypes[layer_num] in ['ReLU', 'Affine']:

            weights = nn.weights[nn.ffn_counter]

            biases = nn.biases[nn.ffn_counter]

            dims = elina_abstract0_dimension(elina_manager, abstraction_shape)

            num_in_pixels = dims.intdim + dims.realdim

            num_out_pixels = len(weights)  

            dimadd = elina_dimchange_alloc(0, num_out_pixels)  # allocate new Elina dim change

            for i in range(num_out_pixels):
                dimadd.contents.dim[i] = num_in_pixels

            elina_abstract0_add_dimensions(elina_manager, True, abstraction_shape, dimadd, False)

            elina_dimchange_free(dimadd)

            np.ascontiguousarray(weights, dtype=np.double)

            np.ascontiguousarray(biases, dtype=np.double)

            var = num_in_pixels

    

            #--------------------------- handle affine layer---------------------------
                
            bounds_array = np.empty([num_out_pixels,2])  #lb and ub after affine for neurons in layer i
            slope_intercept_array = np.empty([num_out_pixels,2]) #slope & intercept for neurons in layer i
            
#             if layer_num <= apply_linear_solver_to_layer:
                #----Use ELINA on FIRST layers-----#
    
            print('--------------ELINA BEFORE AFFINE----------------')   
            bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)

            for i in range(num_out_pixels):
                print('[{},{}]'.format(bounds[i].contents.inf.contents.val.dbl,
                                      bounds[i].contents.sup.contents.val.dbl))
            for i in range(num_out_pixels): 

                tdim = ElinaDim(var)
                linexpr0 = generate_linexpr0(weights[i], biases[i], num_in_pixels)
                abstraction_shape = elina_abstract0_assign_linexpr_array(elina_manager, True, abstraction_shape, tdim,
                                                                         linexpr0, 1, None)
                var += 1

            dimrem = elina_dimchange_alloc(0, num_in_pixels)

            for i in range(num_in_pixels):
                dimrem.contents.dim[i] = i

            elina_abstract0_remove_dimensions(elina_manager, True, abstraction_shape, dimrem)
            elina_dimchange_free(dimrem)  

            bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)

            for i in range(num_out_pixels):
                bounds_array[i,0] = bounds[i].contents.inf.contents.val.dbl
                bounds_array[i,1] = bounds[i].contents.sup.contents.val.dbl

                slope_intercept_array[i,0] = bounds_array[i,1] / (bounds_array[i,1]-bounds_array[i,0])
                slope_intercept_array[i,1] = -(bounds_array[i,1] * bounds_array[i,0])/ (bounds_array[i,1]-bounds_array[i,0])
            
            
            slope_intercept_for_layer['layer' + str(layer_num)] = slope_intercept_array        
            affine_bounds_for_layer['layer' +str(layer_num)] = bounds_array
#             else:

            
#                 bounds_array = np.dot(weights,solver_bounds_for_layer['layer'+str(layer_num-1)])+biases.reshape(len(biases),1)
#                 bounds_array = np.sort(bounds_array)
#                 slope_intercept_array[:,0] = bounds_array[:,1] / (bounds_array[:,1]-bounds_array[:,0])
#                 slope_intercept_array[:,1] = -(bounds_array[:,1] * bounds_array[:,0])/ (bounds_array[:,1]-bounds_array[:,0])

#                 slope_intercept_for_layer['layer' + str(layer_num)] = slope_intercept_array        
#                 affine_bounds_for_layer['layer' +str(layer_num)] = bounds_array

            print("affine bounds keys  ", affine_bounds_for_layer.keys())
            print("{}".format(affine_bounds_for_layer['layer'+str(layer_num)]))

            #--------------------------- handle RELU ---------------------------

            if (nn.layertypes[layer_num] == 'ReLU'):
                abstraction_shape = relu_box_layerwise(elina_manager, True, abstraction_shape, 0, num_out_pixels)

                bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)  # Read in bounds after ReLU

                element_dim = elina_abstract0_dimension(elina_manager, abstraction_shape)
                    

                if layer_num == apply_linear_solver_to_layer:

                    linear_solver_relu_bounds = np.empty([affine_bounds_for_layer['layer'+str(layer_num)].shape[0],2])
                    for neuron in range(affine_bounds_for_layer['layer'+str(layer_num)].shape[0]):
                        linear_solver_relu_bounds[neuron,0],linear_solver_relu_bounds[neuron,1] = get_bounds_linear_solver(nn,
                                                                                                                           affine_bounds_for_layer,
                                                                                       layer_num,neuron,
                                                                                       slope_intercept_for_layer)
                    print('---------------RELU--------------')

                    print('\n Solver output layer: {}\n {}\n'.format(layer_num,linear_solver_relu_bounds))

                    solver_bounds_for_layer['layer'+str(layer_num)] = linear_solver_relu_bounds
                    
                    #--------Update ELINA abstraction shape
                    interval_array = elina_interval_array_alloc(num_out_pixels)
                    for i in range(solver_bounds_for_layer['layer'+str(layer_num)].shape[0]):  

                        elina_interval_set_double(interval_array[i], 
                                                  solver_bounds_for_layer['layer'+str(layer_num)][i,0], 
                                                  solver_bounds_for_layer['layer'+str(layer_num)][i,1])
                       
                        ## construct input abstraction
                    abstraction_shape = elina_abstract0_of_box(elina_manager, 0, num_out_pixels, interval_array)  
                    bounds = elina_abstract0_to_box(elina_manager, abstraction_shape)

#                     print('--------------ELINA updated bounds----------------')    
#                     for i in range(num_out_pixels):
#                         print('[{},{}]'.format(bounds[i].contents.inf.contents.val.dbl,
#                                               bounds[i].contents.sup.contents.val.dbl))

#                 elif layer_num > apply_linear_solver_to_layer:
#                     #---------------Manually propagate till output---------------------#
#                     solver_bounds_for_layer['layer'+str(layer_num)] = affine_bounds_for_layer['layer'+str(layer_num)]
#                     for i in range(affine_bounds_for_layer['layer' +str(layer_num)].shape[0]):
                        
#                         lb = affine_bounds_for_layer['layer' +str(layer_num)][i,0]
#                         ub = affine_bounds_for_layer['layer' +str(layer_num)][i,1]
                        
#                         if (lb <= 0):
#                             solver_bounds_for_layer['layer' +str(layer_num)][i,0] = 0
                        
#                         if (ub<=0):
#                             solver_bounds_for_layer['layer' +str(layer_num)][i,1] = 0
                            
                   

                            
            nn.ffn_counter += 1


        else:

            print(' net type not supported')


    verified_flag = True

    predicted_label = 0

#     output_layer_prob = solver_bounds_for_layer['layer'+str(numlayer-1)]
#     output_layer_prob = bounds[label].contents.inf.contents.val.dbl
    print('--------------ELINA updated LAST LAYER bounds----------------')    
    for i in range(num_out_pixels):
        print('[{},{}]'.format(bounds[i].contents.inf.contents.val.dbl,
                              bounds[i].contents.sup.contents.val.dbl))
#     inf = output_layer_prob[label,0]
    inf = bounds[label].contents.inf.contents.val.dbl
#     print("\Last layer:{} ".format(solver_bounds_for_layer))
    print("\nlabel:{} inf:{}".format(label,inf))

    for j in range(num_out_pixels):

        if (j != label):
            sup = bounds[j].contents.sup.contents.val.dbl
#             sup = output_layer_prob[j,1]
            print("\nsup:{}".format(sup))

            if (inf <= sup):
                predicted_label = label

                verified_flag = False

                break


    elina_abstract0_free(elina_manager, abstraction_shape)

    elina_manager_free(elina_manager)

    return verified_flag

In [6]:
def main(argv, solver_layer):
    logging.basicConfig(level=logging.DEBUG)

    print("\n--------------START--------------------------------")

    if len(argv) < 3 or len(argv) > 4:
        print('usage: python3.6 ' + argv[0] + ' net.txt spec.txt [timeout]')

        exit(1)

    netname = argv[1]
    specname = argv[2]
    epsilon = float(argv[3])

        # c_label = int(argv[4])

    with open(netname, 'r') as netfile:
        netstring = netfile.read()

    with open(specname, 'r') as specfile:
        specstring = specfile.read()

    nn = parse_net(netstring)
    weights = nn.weights
    bias = nn.biases
    print("weights:,",nn.weights[0].shape)
    print("bias,",nn.biases[0].shape)

    x0_low, x0_high = parse_spec(specstring)

    LB_N0, UB_N0 = get_perturbed_image(x0_low, 0)

    print("Analyze to get label.....................")

    label = predict_label(nn,LB_N0,UB_N0,0)

    start = time.time()

    if (label == int(x0_low[0])):

        LB_N0, UB_N0 = get_perturbed_image(x0_low, epsilon)

        print("\nAnalyze to verify robustness.....................")

        verified_flag = verify(nn,LB_N0,UB_N0,label, apply_linear_solver_to_layer= solver_layer)
        
        image_verified = ""
        if (verified_flag):

#             print("verified")
            image_verified = "verified"

        else:

#             print("can not be verified")
            image_verified = "can not be verified"
    else:
        image_verified = "image not correctly classified"
#         print(
#             "image not correctly classified by the network. expected label ", int(x0_low[0]), " classified label: ", label)

    end = time.time()
    analysis_time = (end - start)
#     print("analysis time: ", (end - start), " seconds")
    
    return image_verified, analysis_time,label

In [7]:
solver_layer = 3
network = "6_50"
epsilon = 0.01
num_images = 3

In [8]:
output_file = open("output_"+network+"_eps_"+str(epsilon)+"_solver+ELINA_"+str(solver_layer)+"_numimages_"+str(num_images)+".txt", "w+")
verified_count = 0
for i in range(num_images):
    image_n = "../mnist_images/img"+str(i)+".txt"
    argv = ['python3' ,'../mnist_nets/mnist_relu_'+str(network)+'.txt', image_n, epsilon]
    verified, analysis_time,label = main(argv, solver_layer)
    print(verified)
    if verified == "verified":
        verified_count+=1
    output_file.write("img {} ------------{} ------------label {}-----------------analysis time:{}\n".format(i,
                                                                                                             verified, 
                                                                                                             label,
                                                                                                             analysis_time))
output_file.write("#verified:{}".format(verified_count))   
output_file.close()



--------------START--------------------------------


DEBUG:root:Layers #:6


weights:, (50, 784)
bias, (50,)
Analyze to get label.....................
original num_pixels: 784 , num_in_pixels: 784, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:10
---------------RELU--------------

Analyze to verify robustness.....................
--------------ELINA BEFORE AFFINE----------------
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,

INFO:gurobipy.gurobipy:Academic license - for non-commercial use only


---------------RELU--------------

 Solver output layer: 3
 [[ 2.66057773  4.8259356 ]
 [ 0.          0.95647076]
 [ 0.27186334  1.21089118]
 [ 0.          1.15058457]
 [ 0.          0.83809151]
 [ 0.         -0.        ]
 [ 0.          0.47259765]
 [ 0.         -0.        ]
 [ 0.          1.16794968]
 [ 0.          0.47553178]
 [ 0.          0.46478007]
 [ 0.          0.47031702]
 [ 1.22422587  2.48364177]
 [ 0.         -0.        ]
 [ 0.          1.24851588]
 [ 0.          0.50692791]
 [ 0.02241445  0.78233352]
 [ 0.          0.86043237]
 [ 0.          0.00837176]
 [ 0.67271111  1.92189282]
 [ 0.          0.71407259]
 [ 0.          0.06206864]
 [ 1.31768594  2.7079368 ]
 [ 0.49225463  1.68579069]
 [ 0.          0.63694924]
 [ 1.14751866  2.95617325]
 [ 0.         -0.        ]
 [ 0.         -0.        ]
 [ 0.49414659  1.47955931]
 [ 0.          0.48524255]
 [ 0.          0.40070175]
 [ 0.5904951   1.34796629]
 [ 1.19566123  2.77171707]
 [ 0.         -0.        ]
 [ 0.          0.55015

...

weights:, (50, 784)
bias, (50,)
Analyze to get label.....................
original num_pixels: 784 , num_in_pixels: 784, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:10
---------------RELU--------------

Analyze to verify robustness.....................
--------------ELINA BEFORE AFFINE----------------
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,

---------------RELU--------------

 Solver output layer: 3
 [[ 0.          0.95667746]
 [ 0.          0.9031228 ]
 [ 0.          0.3962352 ]
 [ 5.0579858   7.4376361 ]
 [ 0.          1.45583023]
 [ 0.          0.38486997]
 [ 0.         -0.        ]
 [ 0.          0.62808638]
 [ 0.         -0.        ]
 [ 0.01323839  1.90885273]
 [ 0.          0.05454932]
 [ 0.          0.09361246]
 [ 0.          0.72132904]
 [ 0.66461741  2.0731052 ]
 [ 3.17810992  4.86608085]
 [ 0.          0.10359874]
 [ 0.         -0.        ]
 [ 0.         -0.        ]
 [ 0.17903785  1.76071352]
 [ 0.         -0.        ]
 [ 0.         -0.        ]
 [ 0.          0.56169707]
 [ 0.          2.02380692]
 [ 0.         -0.        ]
 [ 0.         -0.        ]
 [ 5.19328015  7.92962769]
 [ 0.          0.82172018]
 [ 0.          0.48543726]
 [ 1.32931177  2.52148382]
 [ 0.          1.95479003]
 [ 0.25752949  1.65776503]
 [ 0.          1.22926149]
 [ 0.          1.4471755 ]
 [ 0.          0.7081106 ]
 [ 0.          1.67337

DEBUG:root:Layers #:6


weights:, (50, 784)
bias, (50,)
Analyze to get label.....................
original num_pixels: 784 , num_in_pixels: 784, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:50
---------------RELU--------------
original num_pixels: 784 , num_in_pixels: 50, num_out_pixels:10
---------------RELU--------------

Analyze to verify robustness.....................
--------------ELINA BEFORE AFFINE----------------
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,0.01]
[0.0,

---------------RELU--------------

 Solver output layer: 3
 [[0.         2.58707193]
 [0.         2.50116474]
 [0.         2.10967898]
 [0.86509834 5.88494948]
 [0.         1.80773436]
 [0.         2.08214594]
 [0.         1.30918118]
 [0.         1.33631458]
 [0.         2.03699952]
 [0.2459258  4.79771265]
 [0.         0.71635821]
 [0.         1.32238549]
 [0.         2.25166406]
 [0.         2.0896341 ]
 [0.         2.68019191]
 [0.         1.33341379]
 [0.         1.01103285]
 [0.         0.85998343]
 [0.         1.77937976]
 [0.         1.41981965]
 [0.         1.07554134]
 [0.         1.32622441]
 [0.89155941 5.65455012]
 [0.         1.30633194]
 [0.         2.07813981]
 [0.11633457 5.15050353]
 [0.         1.41966825]
 [0.         0.81375104]
 [0.         2.2531375 ]
 [1.41486173 6.45870036]
 [0.         1.98855335]
 [0.44622056 3.79766136]
 [0.31407915 5.22470365]
 [0.         1.50724978]
 [0.         2.4028208 ]
 [0.         2.06387907]
 [0.         1.4320775 ]
 [0.         1.

In [9]:
verified_count

0